# 鉱物ライブラリの作成

---
コマンドプロンプトは、’！’を先頭につけると実行可能

In [2]:
!pip3 install pandas
!pip3 install xlrd
!pip3 install openpyxl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 2.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 4.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


---
キーワードを含む行を抽出

In [ ]:
import pandas as pd

# Excelを読み込み
df = pd.read_excel('./RelabDatabase2022Dec31/catalogues/Sample_Catalogue.xls')

# キーワードを指定
keyword = 'Mineral'

# キーワードを含む行を抽出
extract_df = df[df['GeneralType1'].str.contains(keyword, na=False)]

# 結果をExcelファイルとして出力
extract_df.to_excel('Sample_Catalogue_Mineral.xlsx', index=False)

---
リストにあるキーワードを含む行を抽出

In [29]:
import pandas as pd

# Excelを読み込み
df1 = pd.read_excel('Sample_Catalogue_Mineral.xlsx')

# データフレームをリスト化
list_df1 = df1['SampleID'].to_list()

# Excelを読み込み
df2 = pd.read_excel('./RelabDatabase2022Dec31/catalogues/Spectra_Catalogue.xls')

# リストにあるキーワードを含む行を抽出
extract_df2 = df2.query('SampleID in @list_df1')

# 結果をExcelファイルとして出力
extract_df2.to_excel('Spectra_Catalogue_Mineral.xlsx', index=False)

---
キーワードが完全一致する行を抽出

In [31]:
import pandas as pd

# Excelを読み込み
df = pd.read_excel('Spectra_Catalogue_Mineral.xlsx')

# キーワードが完全一致する行を抽出
extract_df = df[df['SpecCode'] == 'BD-VNIR']

# 結果をExcelファイルとして出力
extract_df.to_excel('Spectra_Catalogue_Mineral_BD-VNIR.xlsx', index=False)

---
ascファイルからバンド部分を抽出

In [114]:
import numpy as np

# float型を小数点表示で固定
np.set_printoptions(suppress=True)

# ascを読み込み
with open('./RelabDatabase2022Dec31/data/a1s/aa/c1aa01.asc') as f:
    # 行区切りで2次元配列化
    l = [item.split() for item in f.readlines()]

    # 1行目にバンド数
    bands = int(l[0][0])

    # バンド部分を抽出、float型に変更
    data_str = np.array(l[1:bands+1])
    data_float = np.asfarray(data_str, dtype=float)

    # 列分解、波長の単位をμmに変更
    wav = data_float[:, 0] / 1000
    ref = data_float[:, 1]

    # リスト結合、1行の2次元配列化（データを横持ちにするため）
    data = np.r_[wav, ref]
    data = np.append(bands, data)
    data = data.reshape(1, len(data))

    # 結果をCSVファイルとして出力
    np.savetxt('test_from_asc.csv', data, delimiter=',', fmt='%.5f')


---
各ascファイルからバンド部分を抽出、収集

In [3]:
import pandas as pd
import numpy as np
import csv

# float型を小数点表示で固定
np.set_printoptions(suppress=True)

# Excelを読み込み
df = pd.read_excel('Spectra_Catalogue_Mineral_BD-VNIR.xlsx')

# データフレームをリスト化、小文字化
list_sample = df['SampleID'].to_list()
list_spectrum = df['SpectrumID'].to_list()
list_sample, list_spectrum = [i.lower() for i in list_sample], [j.lower() for j in list_spectrum]

count = 0

for sample, spectrum in zip(list_sample, list_spectrum):
    # 変数の文字列を分解、リスト化
    l_sample = sample.split('-')

    # ascファイル名を生成
    asc_file = './RelabDatabase2022Dec31/data/{0}/{1}/{2}.asc'.format(l_sample[1], l_sample[0], spectrum)

    # ascを読み込み
    with open(asc_file) as f:
        # 行区切りでリスト型の2次元配列化
        l = [item.split() for item in f.readlines()]

        # 1行目のバンド数抽出
        bands = int(l[0][0])

        # ascファイルのwavとrefを抽出、リスト型の2次元配列（行数：バンド数、列数：wavとrefの2列）
        l = [row[0:2] for row in l[1:bands+1]]
       
        # ndarrayに変換（float型）
        data_float = np.array(l, dtype=float)

        # 列分解、波長の単位をμmに変更
        wav = data_float[:, 0] / 1000
        ref = data_float[:, 1]

        # ndarray結合（データ構造は横持ち）
        data = np.r_[wav, ref]

        # 他の単体データの追加
        data = np.append(bands, data)
        data = np.append(spectrum, data)
        data = np.append(sample, data)

        # 1行のリスト型の2次元配列化
        data = data.reshape(1, len(data)).tolist()

        if count == 0:
            data2 = data
            count = 1
        else:
            # 各ファイルデータを縦持ちで結合
            data2.extend(data)

# 結果をCSVファイルとして出力
with open('./RelabDatabase2022Dec31/Library_Mineral_BD-VNIR.csv', 'w') as f:
    w = csv.writer(f, lineterminator='\n')
    w.writerows(data2)